# Inference notebook (only for submissions)

In [1]:
! pip install ../input/efficientnet-pytorch

Processing /kaggle/input/efficientnet-pytorch
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.0-py3-none-any.whl size=20193 sha256=6d4b9190886bd7819c2bc0b3021a591a817f07decb1aa6571b178cf49dee2fee
  Stored in directory: /root/.cache/pip/wheels/c4/11/46/c78f049f947bbd6a0dc92e74951904c9b2fa9192864ed474dd
Successfully built efficientnet-pytorch


In [2]:
from fastai.vision.all import *
from efficientnet_pytorch import EfficientNet

In [3]:
# this is only for submissions

path_str = '../input/cassava-leaf-disease-classification'
submission_df = pd.read_csv(path_str + '/sample_submission.csv')
test_images_path = path_str + '/test_images/'

In [4]:
class AlbumentationsTransform(RandTransform):
    split_idx,order = None, 2
    
    def __init__(self, train_aug, valid_aug): 
        store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)


def get_train_aug(size): 
    return albumentations.Compose([
            #albumentations.Resize(size, size), # keep this?
            albumentations.RandomResizedCrop(size,size),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)
])

def get_valid_aug(size): 
    return albumentations.Compose([
        albumentations.CenterCrop(size, size, p=1.),
        albumentations.Resize(size, size),
], p=1.)

def get_x(row): return images_path/row['image_id']
def get_y(row): return row['label']

In [5]:
learn = load_learner(Path('../input/effnetmodels/model-f0.pkl'), cpu=False).to_fp16()
learn1 = load_learner(Path('../input/effnetmodels/model-f1.pkl'), cpu=False).to_fp16()
learn2 = load_learner(Path('../input/effnetmodels/model-f2.pkl'), cpu=False).to_fp16()

In [6]:
test_data_path = submission_df['image_id'].apply(lambda x: test_images_path+x)

learn_tst_dl = learn.dls.test_dl(test_data_path)
learn1_tst_dl = learn1.dls.test_dl(test_data_path)
learn2_tst_dl = learn2.dls.test_dl(test_data_path)

In [7]:
## TODO: add item_tfms to .tta
learn_predictions = learn.get_preds(dl=learn_tst_dl)
learn1_predictions = learn1.get_preds(dl=learn1_tst_dl)
learn2_predictions = learn2.get_preds(dl=learn2_tst_dl)

predictions = (learn_predictions[0] + learn1_predictions[0] + learn2_predictions[0]) / 3

In [8]:
submission_df['label'] = np.argmax(predictions, axis=1)

submission_df.to_csv('submission.csv', index=False)